# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

L = 20.0
domain = mesh.create_box(
    MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron
)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim,)))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two

In [3]:
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(
    domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets]
)

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [6]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$,
and create the expression for the first Piola-Kirchhoff stress:

Elasticity parameters

In [9]:
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

Stored strain energy density (compressible neo-Hookean model)

In [10]:
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J)) ** 2

Hyper-elasticity

In [11]:
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity,
the following lines can be uncommented to solve the linear elasticity problem.
```

In [12]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2.
We set the quadrature degree for the integrals to 4.

In [13]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

Define the residual of the equation (we want to find u such that residual(u) = 0)

In [14]:
residual = (
    ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)
)

As the varitional form is non-linear and written on residual form,
we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [15]:
import dolfinx, sys, petsc4py
print("dolfinx version:", dolfinx.__version__)
print("dolfinx path:", dolfinx.__file__)
print("Python:", sys.version)
print("PETSc4py:", petsc4py.__version__)

dolfinx version: 0.9.0
dolfinx path: /home/ug/miniconda3/envs/fenicsx/lib/python3.13/site-packages/dolfinx/__init__.py
Python: 3.13.9 | packaged by conda-forge | (main, Oct 16 2025, 10:31:39) [GCC 14.3.0]
PETSc4py: 3.24.0


In [16]:
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from petsc4py import PETSc

# 1️⃣ Define nonlinear problem
problem = NonlinearProblem(residual, u, bcs=bcs)

# 2️⃣ Configure PETSc options globally
opts = PETSc.Options()
opts["snes_type"] = "newtonls"
opts["snes_linesearch_type"] = "basic"
opts["ksp_type"] = "preonly"
opts["pc_type"] = "lu"
opts["pc_factor_mat_solver_type"] = "petsc"
# You can add monitors if desired:
# opts["snes_monitor"] = ""

# 3️⃣ Create solver and tune tolerances
solver = NewtonSolver(u.function_space.mesh.comm, problem)
solver.rtol = 1e-8
solver.atol = 1e-8
solver.stol = 1e-8
solver.max_it = 50
solver.report = True
solver.convergence_criterion = "incremental"

# 4️⃣ Solve
n_iter, converged = solver.solve(u)
print(f"Converged: {converged}, iterations = {n_iter}")


Converged: True, iterations = 2


We create a function to plot the solution at each time step.

In [17]:
import pyvista
from dolfinx import plot

plotter = pyvista.Plotter()

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, : len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement (|u|) and visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)

# --- Option 1: robust projection method (recommended) ---
u_magnitude_expr = ufl.sqrt(ufl.inner(u, u))

v = ufl.TestFunction(Vs)
u_trial = ufl.TrialFunction(Vs)
a_proj = ufl.inner(u_trial, v) * ufl.dx
L_proj = ufl.inner(u_magnitude_expr, v) * ufl.dx

problem_proj = fem.petsc.LinearProblem(a_proj, L_proj)
magnitude = problem_proj.solve()

# --- Add magnitude to PyVista object ---
warped["mag"] = magnitude.x.array


Finally, we solve the problem over several time steps, updating the z-component of the traction

In [19]:
from dolfinx.io import XDMFFile
import numpy as np
import traceback
from dolfinx import fem

# --- Open XDMF file once and keep it open ---
with XDMFFile(domain.comm, "results_hyperelastic.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)

    tval0 = -1.5

    # --- Loop over load steps ---
    for n in range(1, 20):
        print(f"\n=== Load step {n} ===")
        T.value[2] = n * tval0

        try:
            n_iter, converged = solver.solve(u)
            print(f"Step {n}: {'✅ converged' if converged else '❌ not converged'} in {n_iter} iterations")
        except Exception as e:
            print(f"❌ Solver crashed at step {n}: {e}")
            traceback.print_exc()
            break

        # --- Write displacement field (interpolated to linear space for ParaView) ---
        V_out = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
        u_out = fem.Function(V_out)
        u_out.interpolate(u)
        xdmf.write_function(u_out, n)  # n used as time value
        print(f"📤 Saved step {n} to XDMF (time = {n})")

        # --- Optional: safety check ---
        if np.isnan(u.x.array).any():
            print("⚠️ NaN detected in solution, stopping loop.")
            break

print("✅ Simulation completed successfully.")



=== Load step 1 ===
Step 1: ✅ converged in 2 iterations
📤 Saved step 1 to XDMF (time = 1)

=== Load step 2 ===
Step 2: ✅ converged in 9 iterations
📤 Saved step 2 to XDMF (time = 2)

=== Load step 3 ===
Step 3: ✅ converged in 10 iterations
📤 Saved step 3 to XDMF (time = 3)

=== Load step 4 ===
Step 4: ✅ converged in 9 iterations
📤 Saved step 4 to XDMF (time = 4)

=== Load step 5 ===
Step 5: ✅ converged in 8 iterations
📤 Saved step 5 to XDMF (time = 5)

=== Load step 6 ===
Step 6: ✅ converged in 7 iterations
📤 Saved step 6 to XDMF (time = 6)

=== Load step 7 ===
Step 7: ✅ converged in 6 iterations
📤 Saved step 7 to XDMF (time = 7)

=== Load step 8 ===
Step 8: ✅ converged in 6 iterations
📤 Saved step 8 to XDMF (time = 8)

=== Load step 9 ===
Step 9: ✅ converged in 6 iterations
📤 Saved step 9 to XDMF (time = 9)

=== Load step 10 ===
Step 10: ✅ converged in 6 iterations
📤 Saved step 10 to XDMF (time = 10)

=== Load step 11 ===
Step 11: ✅ converged in 6 iterations
📤 Saved step 11 to XDMF (t